In [1]:
import pandas as pd
import numpy as np
import os

pd.set_option('display.max_columns', None)

# Import Virtual Mode Data

In [2]:
virtual = pd.read_csv("../../data/raw_data/Texas_Schools_LearningModelData_Final.csv", thousands=',')
virtual.head()

,StateName,StateAbbrev,DataLevel,Charter,SchoolName,SchoolType,NCESSchoolID,StateAssignedSchoolID,DistrictName,DistrictType,NCESDistrictID,StateAssignedDistrictID,EnrollmentTotal,TimePeriodInterval,TimePeriodStart,TimePeriodEnd,LearningModel,LearningModelGrK5,LearningModelGr68,LearningModelGr912,LearningModelStateCat,LearningModelStateCatGrK5,LearningModelStateCatGr68,LearningModelStateCatGr912,EnrollmentInPerson,EnrollmentHybrid,EnrollmentVirtual,StaffCount,StaffCountInPerson
0,Texas,TX,School,No,Crosbyton Elementary,Regular school,480000101145,54901101,Crosbyton CISD,Regular local school district,4800001,54901,138.0,Bi-annually,9/1/20,12/31/20,In-person,NaN,NaN,NaN,NaN,NaN,NaN,NaN,125.0,0.0,13.0,NaN,NaN
1,Texas,TX,School,No,Crosbyton Elementary,Regular school,480000101145,54901101,Crosbyton CISD,Regular local school district,4800001,54901,143.0,Bi-annually,1/1/21,6/1/21,In-person,NaN,NaN,NaN,NaN,NaN,NaN,NaN,141.0,0.0,2.0,NaN,NaN
2,Texas,TX,School,No,Crosbyton Secondary,Regular school,480000101146,54901001,Crosbyton CISD,Regular local school district,4800001,54901,171.0,Bi-annually,9/1/20,12/31/20,In-person,NaN,NaN,NaN,NaN,NaN,NaN,NaN,161.0,0.0,10.0,NaN,NaN
3,Texas,TX,School,No,Crosbyton Secondary,Regular school,480000101146,54901001,Crosbyton CISD,Regular local school district,4800001,54901,169.0,Bi-annually,1/1/21,6/1/21,In-person,NaN,NaN,NaN,NaN,NaN,NaN,NaN,165.0,0.0,4.0,NaN,NaN
4,Texas,TX,School,No,Sp Ed Co-Op,Other/alternative school,480000103621,54901200,Crosbyton CISD,Regular local school district,4800001,54901,7.0,Bi-annually,9/1/20,12/31/20,In-person,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,0.0,0.0,NaN,NaN


In [3]:
virtual = pd.read_csv("../../data/raw_data/Texas_Schools_LearningModelData_Final.csv", thousands=',')

virtual['Charter'] = virtual['Charter'].replace({'Yes': 1, 'No': 0})

virtual = virtual[virtual['TimePeriodStart'].str.endswith(('21', '22'))]

# Convert the date columns to datetime format.
virtual['TimePeriodStart'] = pd.to_datetime(virtual['TimePeriodStart'])
virtual['TimePeriodEnd'] = pd.to_datetime(virtual['TimePeriodEnd'])

# Create a new column for year
virtual['YEAR'] = virtual['TimePeriodStart'].dt.year

# Fill in any missing values in LearningModel with 'InPerson'
virtual['LearningModel'] = virtual['LearningModel'].fillna('InPerson')

# Replace 'In-person' with 'InPerson'
virtual['LearningModel'] = virtual['LearningModel'].replace('In-person', 'InPerson')

# Calculate the number of days for each row
virtual['Days'] = (virtual['TimePeriodEnd'] - virtual['TimePeriodStart']).dt.days

# Group by School, Year, LearningModel, and Charter and sum the number of days
grouped = virtual.groupby(['StateAssignedSchoolID', 'YEAR', 'LearningModel', 'Charter'])['Days'].sum().reset_index()

# Pivot the data so we have separate columns for each learning model
pivot = grouped.pivot_table(index=['StateAssignedSchoolID', 'YEAR', 'Charter'], columns='LearningModel', values='Days', fill_value=0)

# Reset the index
pivot.reset_index(inplace=True)

# Calculate the total days in each year
pivot['TotalDays'] = pivot['Virtual'] + pivot['InPerson']

# Calculate the percentage of days that are virtual and hybrid for each year
pivot['VirtualPercent'] = pivot['Virtual'] / pivot['TotalDays']

# Manually create a 'HybridPercent' column and set it to zero
pivot['HybridPercent'] = 0  # This line added

# Calculate the score for each year
pivot['Score'] = pivot['VirtualPercent']

# Reset the column names after pivot
pivot.columns.name = None

In [4]:
common_ids = set(pivot["StateAssignedSchoolID"])

# Import Dropout Data

In [ ]:
### Regressor Data
data21 = pd.read_excel("../../data/raw_data/campus-data-download-drop-2021.xlsx", thousands=',', sheet_name=2)
data19 = pd.read_excel("../../data/raw_data/campus-data-download-drop-1819.xlsx", thousands=',', sheet_name=2)
data18 = pd.read_excel("../../data/raw_data/campus-data-download-drop-1718.xlsx", thousands=',', sheet_name=2)
data17 = pd.read_excel("../../data/raw_data/campus-data-download-drop-1617.xlsx", thousands=',', sheet_name=2)
data16 = pd.read_excel("../../data/raw_data/campus-data-download-drop-1516.xlsx", thousands=',', sheet_name=2)
data15 = pd.read_excel("../../data/raw_data/campus-data-download-drop-1415.xlsx", thousands=',', sheet_name=2)

data18.head(5)

In [ ]:
data21.head(5)

In [ ]:
common_ids = set(pivot["StateAssignedSchoolID"])

data = [data17, data18, data19, data21]
year = [2017, 2018, 2019, 2021]
# data = [data18, data19, data21]
# year = [2018, 2019, 2021]
calc_columns = ["CAMPUS","CAMP_ALLD","CAMP_ALLR","CAMP_AAD","CAMP_WHD","CAMP_HSD","CAMP_ASD","CAMP_ECND", "DISTRICT"]
final_columns = ["schoolcode","year","totalenrolled","dropout","lowincome","black","white","hispanic","asian", "district"]

for i in range(3):
    data[i].columns = data[i].columns.str.upper()

for i, (d, y) in enumerate(zip(data, year)):
    data[i].columns = data[i].columns.str.upper()
    data[i] = data[i][data[i]['GRADESPAN'] == 912]    
    data[i] = data[i][data[i]['CALC_FOR_STATE_ACCT'] == "Yes"]
    data[i] = data[i][calc_columns]
    mask = data[i].applymap(lambda x: '<' in str(x)).any(axis=1)
    data[i].replace(".", 0, inplace=True)
    data[i] = data[i][~mask]

# for i, (d, y) in enumerate(zip(data, year)):
# common_ids = common_ids & set(data[0]['CAMPUS']) & set(data[1]['CAMPUS']) & set(data[2]['CAMPUS'])

for i in range(len(data)):
    data[i] = data[i][data[i]['CAMPUS'].isin(common_ids)]

for i, (d, y) in enumerate(zip(data, year)):
    data[i].replace(0.0, 0, inplace=True)
    data[i]["schoolcode"] = data[i]["CAMPUS"].astype(float).astype(int)
    data[i]["totalenrolled"] = data[i]["CAMP_ALLD"].astype(float).astype(int)
    data[i]["dropout"] = data[i]["CAMP_ALLR"].astype(float).astype(int)
    print([min(data[i]["dropout"]), max(data[i]["dropout"])])
    data[i]["black"] = data[i]["CAMP_AAD"].astype(float).astype(int) / data[i]["CAMP_ALLD"].astype(float).astype(int)
    data[i]["white"] = data[i]["CAMP_WHD"].astype(float).astype(int) / data[i]["CAMP_ALLD"].astype(float).astype(int)
    data[i]["hispanic"] = data[i]["CAMP_HSD"].astype(float).astype(int) / data[i]["CAMP_ALLD"].astype(float).astype(int)
    data[i]["asian"] = data[i]["CAMP_ASD"].astype(float).astype(int) / data[i]["CAMP_ALLD"].astype(float).astype(int)
    data[i]["lowincome"] = data[i]["CAMP_ECND"].astype(float).astype(int) / data[i]["CAMP_ALLD"].astype(float).astype(int)
    data[i]["district"] = data[i]["DISTRICT"]
    
    
for i, (d, y) in enumerate(zip(data, year)):
    d['year'] = y
    data[i] = data[i][final_columns]

In [ ]:
pivot = pivot.drop(columns=['InPerson', 'Virtual', 'TotalDays'])
pivot

In [ ]:
# Concatenate all the dataframes
final_data = pd.concat(data)


In [ ]:
problematic_cols = [
 'lowincome',
  'dropout',
 'totalenrolled',
      ]

final_data = final_data[~final_data[problematic_cols].lt(0).any(axis=1)]

In [ ]:
# Merge dropout_df with final_data
final_data = pd.merge(final_data, pivot, left_on=['schoolcode', 'year'], right_on=['StateAssignedSchoolID', 'YEAR'], how='left')

In [ ]:
final_data['Charter'] = final_data.groupby('schoolcode')['Charter'].bfill()


final_data = final_data.fillna(0)
final_data = final_data.drop(columns=["StateAssignedSchoolID", "YEAR"])
# Rename multiple columns
final_data = final_data.rename(columns={               
                        'VirtualPercent': 'virtualper',    
                        'HybridPercent': 'hybridper',    
                        'Score': 'schoolmode',
                        'Charter': 'charter'
                                       })

final_cols = final_data.columns.tolist()
final_cols

In [ ]:
final_cols = [
'schoolcode',
'district',
 'year',
 'charter',
 'dropout',
 'schoolmode',
 'virtualper',
 'hybridper',
 'totalenrolled',
 'lowincome',
 'white',
 'black',
 'hispanic',
 'asian',
      ]

multi_cols = [
'schoolcode',
'district',
 'year',
 'charter',
 'dropout',
 'totalenrolled',
 'lowincome',
 'white',
 'black',
 'hispanic',
 'asian',
      ]

In [ ]:
final_data = final_data[final_cols]
for column in multi_cols[5:]:
    if column != "totalenrolled":
        final_data[column] = final_data[column] * 100

In [ ]:
problematic_cols = [
 'white',
 'black',
 'hispanic',
 'asian',
      ]

rounding_cols = [
 'schoolmode',
 'virtualper',
 'hybridper',
 'totalenrolled',
 'lowincome',
 'white',
 'black',
 'hispanic',
 'asian',
      ]

final_data[problematic_cols] = final_data[problematic_cols].abs()
final_data[rounding_cols] = final_data[rounding_cols].round(4)

In [ ]:
# print(final_data["dropout"].value_counts().to_dict())
# print(final_data["dropout"].value_counts().to_dict())
export_dataframe = final_data

export_dataframe["schoolcode"] = export_dataframe["schoolcode"].astype(int).astype(str)

export_dataframe["state"] = "tx"
export_dataframe["schoolcode"] = export_dataframe["schoolcode"].str.zfill(9)
export_dataframe["countycode"] = export_dataframe["schoolcode"].str[:3].astype(str)
export_dataframe["districtcode"] = export_dataframe["schoolcode"].str[:6].astype(str)
export_dataframe['districtcode'] = export_dataframe['districtcode'].astype(str)
export_dataframe['countycode'] = export_dataframe['countycode'].astype(str)

export_dataframe["schoolcode"] = export_dataframe["schoolcode"] + "tx"
export_dataframe["districtcode"] = export_dataframe["districtcode"] + "tx"
export_dataframe["countycode"] = export_dataframe["countycode"] + "tx"

final_cols = [
'schoolcode',
"districtcode",
"countycode",
 'year',
 'charter',
 'dropout',
 'schoolmode',
 'virtualper',
 'hybridper',
 'totalenrolled',
 'lowincome',
 'white',
 'black',
 'hispanic',
 'state'
      ]

export_dataframe = export_dataframe[final_cols]
export_dataframe.head(10)

# Identify the unique years in the dataset.
unique_years = set(export_dataframe['year'])

# Group the data by ENTITY_CD and get the unique years for each group.
entity_groups = export_dataframe.groupby('schoolcode')['year'].unique().reset_index()

# Find the ENTITY_CD values which have all the unique years.
valid_entity_cd = entity_groups[entity_groups['year'].apply(lambda x: set(x) == unique_years)]['schoolcode']

# Filter the main data for these ENTITY_CD values.
export_dataframe = export_dataframe[export_dataframe['schoolcode'].isin(valid_entity_cd)]

# Export Texas Data

In [ ]:
export_dataframe = export_dataframe.sort_values(by=['school', 'year'], ascending=[True, True])
export_dataframe.to_csv("../final_data_component/final_data_tx_dropout.csv")

In [ ]:
problematic_cols = [
 'white',
 'black',
 'hispanic',
 'lowincome',
 'dropout',
 'totalenrolled'
      ]

export_dataframe[export_dataframe[problematic_cols].lt(0).any(axis=1)]